In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import imageio.v3 as iio
from PIL import Image
import numpy as np
import torchmetrics
import config
import models as m
import utils as u

In [6]:
image_size=(160,240)
patch_size=config.pretrain_config['patch_size']
embed_dim=config.pretrain_config['embed_dim']
expander_out=config.pretrain_config['expander_out']
kernel_size=config.finetune_config['kernel_size']
padding=config.finetune_config['padding']
stride=config.finetune_config['stride']
batch_size=config.finetune_config['batch_size']
num_epochs=config.finetune_config['num_epochs']
lr=config.finetune_config['lr']

hidden_folder="Dataset_Student/val/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

VICReg_model_path = config.pretrain_config['model_id']
VICReg_model = m.VICReg(image_size, patch_size, embed_dim, expander_out=expander_out)
VICReg_model.load_state_dict(torch.load(VICReg_model_path))
VICReg_model.eval()
print(f'loaded model {config.pretrain_config["model_id"]}')

final_model_path = 'video_predictor_finetuned_1682558449.pth'
video_prediction_model = m.VideoPredictor(VICReg_model, kernel_size, padding, stride)
video_prediction_model.load_state_dict(torch.load(final_model_path))
video_prediction_model = video_prediction_model.to(device)
video_prediction_model.eval()

hidden_dataloader = u.create_finetune_dataloader(hidden_folder, image_size, batch_size, train_or_val='fake_hidden')

loaded model VICReg_pretrained_1682484312.pth
loading fake_hidden finetuning data from disk...
done


In [7]:
len_data = 0
for data in hidden_dataloader:
    x, y = data
    len_data += x.size(0)
print(len_data)

1000


In [20]:
len_data = 0
for data in hidden_dataloader:
    x, y = data
    len_data += x.size(0)
output = torch.zeros(len_data, 160, 240)

b_index = 0
for data in tqdm(hidden_dataloader, desc='generating outputs'):
    x, y = data
    x = x.to(device)
    y = y.to(device)
    y_pred = video_prediction_model(x)
    y_pred = y_pred.argmax(dim=1)
    output[b_index:b_index+batch_size] = y_pred
    b_index += batch_size

torch.save(output, 'submitted_tensor.pt')


generating outputs: 100%|██████████| 63/63 [00:02<00:00, 21.78it/s]
